# Logic

This Jupyter notebook acts as supporting material for topics covered in __Chapter 6 Logical Agents__, __Chapter 7 First-Order Logic__ and __Chapter 8 Inference in First-Order Logic__ of the book *[Artificial Intelligence: A Modern Approach](http://aima.cs.berkeley.edu)*. We make use of the implementations in the [logic.py](https://github.com/aimacode/aima-python/blob/master/logic.py) module. See the [intro notebook](https://github.com/aimacode/aima-python/blob/master/intro.ipynb) for instructions.

Let's first import everything from the `logic` module.

In [1]:
from utils import *
from logic import *
from notebook import psource

## CONTENTS
- Logical sentences
    - Expr
    - PropKB
    - Knowledge-based agents
    - Inference in propositional knowledge base
        - Truth table enumeration
        - Proof by resolution
        - Forward and backward chaining
        - DPLL
        - WalkSAT
        - SATPlan

## Logical Sentences

The `Expr` class is designed to represent any kind of mathematical expression. The simplest type of `Expr` is a symbol, which can be defined with the function `Symbol`:

In [2]:
Symbol('x')

x

Or we can define multiple symbols at the same time with the function `symbols`:

In [3]:
(x, y, P, Q, f) = symbols('x, y, P, Q, f')

We can combine `Expr`s with the regular Python infix and prefix operators. Here's how we would form the logical sentence "P and not Q":

In [4]:
P & ~Q

(P & ~Q)

This works because the `Expr` class overloads the `&` operator with this definition:

```python
def __and__(self, other): return Expr('&',  self, other)```
     
and does similar overloads for the other operators. An `Expr` has two fields: `op` for the operator, which is always a string, and `args` for the arguments, which is a tuple of 0 or more expressions. By "expression," I mean either an instance of `Expr`, or a number. Let's take a look at the fields for some `Expr` examples:

In [5]:
sentence = P & ~Q

sentence.op

'&'

In [6]:
sentence.args

(P, ~Q)

In [7]:
P.op

'P'

In [8]:
P.args

()

In [9]:
(~Q).op

'~'

In [10]:
(~Q).args

(Q,)

In [11]:
Pxy = P(x, y)

Pxy.op

'P'

In [12]:
Pxy.args

(x, y)

It is important to note that the `Expr` class does not define the *logic* of Propositional Logic sentences; it just gives you a way to *represent* expressions. Think of an `Expr` as an [abstract syntax tree](https://en.wikipedia.org/wiki/Abstract_syntax_tree).  Each of the `args` in an `Expr` can be either a symbol, a number, or a nested `Expr`. We can nest these trees to any depth. Here is a deply nested `Expr`:

In [13]:
3 * f(x, y) + P(y) / 2 + 1

(((3 * f(x, y)) + (P(y) / 2)) + 1)

## Operators for Constructing Logical Sentences

Here is a table of the operators that can be used to form sentences. Note that we have a problem: we want to use Python operators to make sentences, so that our programs (and our interactive sessions like the one here) will show simple code. But Python does not allow implication arrows as operators, so for now we have to use a more verbose notation that Python does allow: `|'==>'|` instead of just `==>`. Alternately, you can always use the more verbose `Expr` constructor forms:

| Operation                | Book | Python Infix Input | Python Output | Python `Expr` Input
|--------------------------|----------------------|-------------------------|---|---|
| Negation                 | &not; P      | `~P`                       | `~P` | `Expr('~', P)`
| And                      | P &and; Q       | `P & Q`                     | `P & Q` | `Expr('&', P, Q)`
| Or                       | P &or; Q | `P`<tt> &#124; </tt>`Q`| `P`<tt> &#124; </tt>`Q` | `Expr('`&#124;`', P, Q)`
| Inequality (Xor)         | P &ne; Q     | `P ^ Q`                | `P ^ Q`  | `Expr('^', P, Q)`
| Implication                  | P &rarr; Q    | `P` <tt>&#124;</tt>`'==>'`<tt>&#124;</tt> `Q`   | `P ==> Q` | `Expr('==>', P, Q)`
| Reverse Implication      | Q &larr; P     | `Q` <tt>&#124;</tt>`'<=='`<tt>&#124;</tt> `P`  |`Q <== P` | `Expr('<==', Q, P)`
| Equivalence            | P &harr; Q   | `P` <tt>&#124;</tt>`'<=>'`<tt>&#124;</tt> `Q`   |`P <=> Q` | `Expr('<=>', P, Q)`

Here's an example of defining a sentence with an implication arrow:

In [14]:
~(P & Q)  |'==>'|  (~P | ~Q)

(~(P & Q) ==> (~P | ~Q))

## `expr`: a Shortcut for Constructing Sentences

If the `|'==>'|` notation looks ugly to you, you can use the function `expr` instead:

In [15]:
expr('~(P & Q)  ==>  (~P | ~Q)')

(~(P & Q) ==> (~P | ~Q))

`expr` takes a string as input, and parses it into an `Expr`. The string can contain arrow operators: `==>`, `<==`, or `<=>`, which are handled as if they were regular Python infix operators. And `expr` automatically defines any symbols, so you don't need to pre-define them:

In [16]:
expr('sqrt(b ** 2 - 4 * a * c)')

sqrt(((b ** 2) - ((4 * a) * c)))

For now that's all you need to know about `expr`. If you are interested, we explain the messy details of how `expr` is implemented and how `|'==>'|` is handled in the appendix.

## Propositional Knowledge Bases: `PropKB`

The class `PropKB` can be used to represent a knowledge base of propositional logic sentences.

We see that the class `KB` has four methods, apart from `__init__`. A point to note here: the `ask` method simply calls the `ask_generator` method. Thus, this one has already been implemented, and what you'll have to actually implement when you create your own knowledge base class (though you'll probably never need to, considering the ones we've created for you) will be the `ask_generator` function and not the `ask` function itself.

The class `PropKB` now.
* `__init__(self, sentence=None)` : The constructor `__init__` creates a single field `clauses` which will be a list of all the sentences of the knowledge base. Note that each one of these sentences will be a 'clause' i.e. a sentence which is made up of only literals and `or`s.
* `tell(self, sentence)` : When you want to add a sentence to the KB, you use the `tell` method. This method takes a sentence, converts it to its CNF, extracts all the clauses, and adds all these clauses to the `clauses` field. So, you need not worry about `tell`ing only clauses to the knowledge base. You can `tell` the knowledge base a sentence in any form that you wish; converting it to CNF and adding the resulting clauses will be handled by the `tell` method.
* `ask_generator(self, query)` : The `ask_generator` function is used by the `ask` function. It calls the `tt_entails` function, which in turn returns `True` if the knowledge base entails query and `False` otherwise. The `ask_generator` itself returns an empty dict `{}` if the knowledge base entails query and `None` otherwise. This might seem a little bit weird to you. After all, it makes more sense just to return a `True` or a `False` instead of the `{}` or `None` But this is done to maintain consistency with the way things are in First-Order Logic, where an `ask_generator` function is supposed to return all the substitutions that make the query true. Hence the dict, to return all these substitutions. I will be mostly be using the `ask` function which returns a `{}` or a `False`, but if you don't like this, you can always use the `ask_if_true` function which returns a `True` or a `False`.
* `retract(self, sentence)` : This function removes all the clauses of the sentence given, from the knowledge base. Like the `tell` function, you don't have to pass clauses to remove them from the knowledge base; any sentence will do fine. The function will take care of converting that sentence to clauses and then remove those.

## Wumpus World KB
Let us create a `PropKB` for the wumpus world with the sentences mentioned in `section 7.4.3`.

In [17]:
wumpus_kb = PropKB()

We define the symbols we use in our clauses.<br/>
$P_{x, y}$ is true if there is a pit in `[x, y]`.<br/>
$B_{x, y}$ is true if the agent senses breeze in `[x, y]`.<br/>

In [18]:
P11, P12, P21, P22, P31, B11, B21 = expr('P11, P12, P21, P22, P31, B11, B21')

Now we tell sentences based on `section 7.4.3`.<br/>
There is no pit in `[1,1]`.

In [19]:
wumpus_kb.tell(~P11)

A square is breezy if and only if there is a pit in a neighboring square. This has to be stated for each square but for now, we include just the relevant squares.

In [20]:
wumpus_kb.tell(B11 | '<=>' | ((P12 | P21)))
wumpus_kb.tell(B21 | '<=>' | ((P11 | P22 | P31)))

Now we include the breeze percepts for the first two squares leading up to the situation in `Figure 7.3(b)`

In [21]:
wumpus_kb.tell(~B11)
wumpus_kb.tell(B21)

We can check the clauses stored in a `KB` by accessing its `clauses` variable

In [22]:
wumpus_kb.clauses

[~P11,
 (~P12 | B11),
 (~P21 | B11),
 (P12 | P21 | ~B11),
 (~P11 | B21),
 (~P22 | B21),
 (~P31 | B21),
 (P11 | P22 | P31 | ~B21),
 ~B11,
 B21]

We see that the equivalence $B_{1, 1} \iff (P_{1, 2} \lor P_{2, 1})$ was automatically converted to two implications which were inturn converted to CNF which is stored in the `KB`.<br/>
$B_{1, 1} \iff (P_{1, 2} \lor P_{2, 1})$ was split into $B_{1, 1} \implies (P_{1, 2} \lor P_{2, 1})$ and $B_{1, 1} \Longleftarrow (P_{1, 2} \lor P_{2, 1})$.<br/>
$B_{1, 1} \implies (P_{1, 2} \lor P_{2, 1})$ was converted to $P_{1, 2} \lor P_{2, 1} \lor \neg B_{1, 1}$.<br/>
$B_{1, 1} \Longleftarrow (P_{1, 2} \lor P_{2, 1})$ was converted to $\neg (P_{1, 2} \lor P_{2, 1}) \lor B_{1, 1}$ which becomes $(\neg P_{1, 2} \lor B_{1, 1}) \land (\neg P_{2, 1} \lor B_{1, 1})$ after applying De Morgan's laws and distributing the disjunction.<br/>
$B_{2, 1} \iff (P_{1, 1} \lor P_{2, 2} \lor P_{3, 2})$ is converted in similar manner.

In [23]:
wumpus_kb.ask(~B11)

{}

In [24]:
wumpus_kb.ask(B11)

False

## Knowledge based agents

A knowledge-based agent is a simple generic agent that maintains and handles a knowledge base.
The knowledge base may initially contain some background knowledge.
<br>
The purpose of a KB agent is to provide a level of abstraction over knowledge-base manipulation and is to be used as a base class for agents that work on a knowledge base.
<br>
Given a percept, the KB agent adds the percept to its knowledge base, asks the knowledge base for the best action, and tells the knowledge base that it has in fact taken that action.
<br>
Our implementation of `KB-Agent` is encapsulated in a class `KBAgentProgram` which inherits from the `KB` class.
<br>
Let's have a look.

In [25]:
psource(KBAgentProgram)

The helper functions `make_percept_sentence`, `make_action_query` and `make_action_sentence` are all aptly named and as expected,
`make_percept_sentence` makes first-order logic sentences about percepts we want our agent to receive,
`make_action_query` asks the underlying `KB` about the action that should be taken and
`make_action_sentence` tells the underlying `KB` about the action it has just taken.

# Authors

This notebook by [Chirag Vartak](https://github.com/chiragvartak) and [Peter Norvig](https://github.com/norvig).

